In [1]:
from free_agent.chain_of_thoughts import CoTAgent

question = "如何构建一个LSTM模型预测股票T+1收益？给出详细的Python代码，使用pytorch框架"

model = "deepseek-chat"

max_iter = 15
temperature = 0.9

agent = CoTAgent(model=model, max_iter=max_iter, temperature=temperature)
answer = agent.get_cot_answer(question)

```markdown
# 你的角色
精通深度学习和金融数据分析的算法工程师

# 你的任务
请你帮我使用PyTorch构建一个LSTM模型，用于预测股票T+1日的收益。

# 背景
我们将使用历史股票价格数据来训练LSTM模型，预测未来一天的股票收益。数据集包括开盘价、收盘价、最高价、最低价和成交量等特征。

# 要求
- 请尽可能将你的每次回答都写得详细一些，不需要考虑token限制；
- 将比较长的，并且可以包装在一起的代码都用函数包装起来。
- 你的每次回答以# 自取标题作为开始，以[Step End]作为结束。
- 当任务最终完成时（也就是最后一个步骤结束后），请发送"[END]"

# 你的工作流
请按照以下步骤一步步进行：
1. 请在建立Python代码之前，给出这个建模过程的所有步骤。
2. 考虑上述步骤有哪些地方可以完善，然后给出完善后的步骤。
3. 根据完善后的步骤，一步一步构建Python代码，请注意，你每次只需要完成一个步骤的代码，当我说继续的时候，你再继续下一步。

现在，请开始：
```
--------------------------------------------------第1轮对话--------------------------------------------------

# 自取标题：建模过程的所有步骤

[Step End]

# 自取标题：完善后的建模步骤

1. 数据预处理：
   - 加载数据集
   - 数据清洗（处理缺失值、异常值等）
   - 特征工程（可能包括对数收益率计算、特征缩放等）
   - 数据分割（训练集和测试集）
   - 创建时间序列数据集（滑动窗口法）

2. 模型构建：
   - 定义LSTM模型结构
   - 初始化模型参数
   - 定义损失函数和优化器

3. 模型训练：
   - 训练模型
   - 监控训练过程中的性能指标（如损失和准确率）
   - 保存最佳模型

4. 模型评估：
   - 在测试集上评估模型性能
   - 分析模型预测结果

5. 模型优化：
   - 调整模型参数
   - 尝试不同的模型结构
   - 应用正则化技术（如Dropout）

6. 结果分析与报告：
   - 生成预测结果图表
   - 分析模型的优缺点
   - 提出改进建议


In [2]:
from tools.llm_api import * # 用于调用llm模型
from tools.json_tool import * # 用于解析json
from tools.code_interpreter import run_code 
from tools.browse import get_url_text
from tools.extract_pdf_text import extract_text_from_pdf

class Agent:
    def __init__(self, angent_name, system_prompt_path, model, temperature):
        self.name = angent_name
        self.model = model
        self.temperature = temperature
        with open(system_prompt_path, "r", encoding="utf-8") as f:
            self.system_prompt = f.read()
        self.conversations = [{"role": "system", "content": self.system_prompt}]
        self.tool_list = [
              "run_code"
            , "read_url"
            , "read_pdf"
        ]   
         
    def get_prompt(self, custom_prompt_path):
        """_summary_

        Args:
            custom_prompt_path (txt): "自定义的提示词路径"

        Returns:
            str: "返回自定义的提示词" 
        """
        with open(f"prompts/{custom_prompt_path}.py", "r", encoding="utf-8") as file:
            prompt = file.read()
        return prompt    
    
    def get_answer(self, question):
        self.conversations.append({"role": "user", "content": question})
        ans = ""
        for char in get_llm_answer_converse(self.conversations, self.model, self.temperature):
            ans += char
            print(char, end='', flush=True)
        self.conversations.append({"role": "assistant", "content": ans})
        return ans

    # 反思 
    def optimize_result(self, question):
        with open(f"free_agent/prompts/cot_optimize_prompt.py", "r", encoding="utf-8") as file:
            cot_optimize_prompt = file.read()
        cot_optimize_prompt = cot_optimize_prompt.format(question)
        ans = self.get_answer(cot_optimize_prompt)
        return ans

    # code interpreter
    def run_code(self, code):
        result = run_code(code)
        return result
    
    # 打开某个页面读取所有文本
    def read_url(self, url):
        result = get_url_text(url)
        return result
    
    # 读取某个PDF的所有文本
    def read_pdf(self, pdf_path):
        """_summary_

        Args:
            pdf_path (str): "PDF文件路径"

        Returns:
            list: "返回PDF的所有文本"
        """
        result = extract_text_from_pdf(pdf_path)
        return result
    
    # 思维链
    def get_cot_ans(self, question):
        self.conversations.append({"role": "user", "content": question})
        agent = CoTAgent(model=model, max_iter=max_iter, temperature=temperature)
        answer = agent.get_cot_answer(question)
        return answer
    

    